# [TorchServe](https://github.com/pytorch/serve)

In [1]:
!git clone https://github.com/pytorch/serve.git

Cloning into 'serve'...
remote: Enumerating objects: 19714, done.
remote: Counting objects: 100% (358/358), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 19714 (delta 187), reused 215 (delta 97), pack-reused 19356
Receiving objects: 100% (19714/19714), 42.57 MiB | 32.01 MiB/s, done.
Resolving deltas: 100% (11386/11386), done.


## Install TorchServe and torch-model-archiver

In [2]:
%cd serve

/content/serve


In [3]:
# For CPU
#!python ./ts_scripts/install_dependencies.py

# For GPU with Cuda 11.0
!python ./ts_scripts/install_dependencies.py --cuda=cu110

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [422 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,148 kB]
Get:12 http://arch

## Serve a model

### [Serving Huggingface Transformers using TorchServe](https://github.com/pytorch/serve/tree/master/examples/Huggingface_Transformers)


In [4]:
%cd /content/serve/examples/Huggingface_Transformers

/content/serve/examples/Huggingface_Transformers


In [5]:
!mkdir model_store

In [7]:
!pip install transformers

     |████████████████████████████████| 2.1 MB 6.5 MB/s 
     |████████████████████████████████| 3.3 MB 23.8 MB/s 
     |████████████████████████████████| 895 kB 47.5 MB/s 


In [13]:
#!perl -pi.bak -e "s|True|true|g" setup_config.json
#!diff setup_config.json.bak setup_config.json
#!mv setup_config.json.bak setup_config.json
!perl -pi.bak -e "s|,do_lower_case=do_lower_case||g" Download_Transformer_models.py
!diff Download_Transformer_models.py.bak Download_Transformer_models.py

22c22
<         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name,do_lower_case=do_lower_case)
---
>         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
26c26
<         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name,do_lower_case=do_lower_case)
---
>         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
30c30
<         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name,do_lower_case=do_lower_case)
---
>         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)


In [37]:
!perl -pi.bak -e "s|, do_lower_case=self.setup_config\[\"do_lower_case\"\]||g;s|do_lower_case=self.setup_config\[\"do_lower_case\"\],\n||g" Transformer_handler_generalized.py
!diff Transformer_handler_generalized.py.bak Transformer_handler_generalized.py
#!mv Transformer_handler_generalized.py.bak Transformer_handler_generalized.py

75c75
<                 model_dir, do_lower_case=self.setup_config["do_lower_case"]
---
>                 model_dir
80,81c80
<                 do_lower_case=self.setup_config["do_lower_case"],
<             )
---
>                             )


In [14]:
!python Download_Transformer_models.py

2021-05-12 05:49:03.592711: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Transformers version 4.5.1
Download model and tokenizer bert-base-uncased
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you exp

In [15]:
!pip install torch-model-archiver

In [38]:
!torch-model-archiver --model-name BERTSeqClassification --version 1.0 --serialized-file Transformer_model/pytorch_model.bin --handler ./Transformer_handler_generalized.py --extra-files "Transformer_model/config.json,./setup_config.json,./Seq_classification_artifacts/index_to_name.json"

In [39]:
!mkdir -p model_store
!mv BERTSeqClassification.mar model_store/

## Start TorchServe to serve the model

In [19]:
!pip install torchserve

     |████████████████████████████████| 17.8 MB 1.4 MB/s 


### [port in use: try to test/run torchserve in colab (Torchserve stopped. java.io.IOException: Failed to bind to address: http://127.0.0.1:8080) #346](https://github.com/pytorch/serve/issues/346#issuecomment-628411129)

In [60]:
%%writefile config.properties
inference_address=http://127.0.0.1:8880
management_address=http://127.0.0.1:8881
metrics_address=http://127.0.0.1:8882

Overwriting config.properties


In [61]:
get_ipython().system_raw(
    'torchserve --start --model-store model_store --models my_tc=BERTSeqClassification.mar --ts-config config.properties &'
)

In [63]:
!curl -X POST http://127.0.0.1:8880/predictions/my_tc -T ./Seq_classification_artifacts/sample_text.txt

Accepted

In [64]:
!curl -X POST http://127.0.0.1:8880/explanations/my_tc -T ./Seq_classification_artifacts/sample_text.txt

{
  "importances": [
    0.0,
    -0.5859627050756424,
    -0.13570754204744964,
    0.10065775319531475,
    -0.09294911424045345,
    -0.25705342798363534,
    0.1452277118361304,
    0.03582634127020333,
    -0.10992920500263736,
    0.22334276596630362,
    0.29781911854376647,
    -0.1423081185606594,
    -0.24605928122455584,
    -0.5473108360394116,
    0.0
  ],
  "words": [
    "[CLS]",
    "bloomberg",
    "has",
    "decided",
    "to",
    "publish",
    "a",
    "new",
    "report",
    "on",
    "the",
    "global",
    "economy",
    ".",
    "[SEP]"
  ],
  "delta": -0.006339518862528232
}

In [71]:
!torchserve --stop

TorchServe has stopped.
